In [1]:
import requests
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import csv
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from nltk import pos_tag
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [2]:
##SCRAPER CODE
# names = []
# reviews = []
# grades = []
# url = 'https://www.temptalia.com/category/reviews/page/1035/'
# response = requests.get(url)
# page = response.text
# soup = BeautifulSoup(page, "lxml")
# product_links = []
# links = soup.find_all(class_='btn btn-secondary btn-lg more-link')
# for link in links:
#     product_links.append(link.get('href'))
# for link in product_links:
#     response = requests.get(link)
#     soup = BeautifulSoup(response.text, "lxml")
#     product_names = soup.find_all(class_ = 'hidden-md-up text-xs-center text-md-left text-uppercase pt-1 mb-2')
#     product_reviews = (soup.find_all('p'))
#     product_grades = (soup.find_all(class_ = 'glossover-grade f-7 bold'))
#     counter = 0
#     for review in product_reviews:
#         if '$' in review.text and len(review.text) > 35:
#             reviews.append(review.text)
#             names.append(product_names[counter].text)
#             grades.append(product_grades[counter].text)
#             counter += 1
# new_df = pd.DataFrame(data = {'names': names, 'reviews': reviews, 'grades': grades})
# new_df.to_csv('temp_sheet.csv')
# new_df = new_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# new_df.tail()

In [3]:
new_df = pd.read_csv('output.csv', encoding = "ISO-8859-1")

In [4]:
new_df.tail()

,Brand,Name,Product,Review,Grade
5168,MAC,OVERSPICED,Lipgloss,"a dirty muted coral. ItÕs a warm-toned, light-...",C+
5169,MAYBELLINE,NUDE ILLUSION (060),Lipgloss,"light-medium peachy beige with soft, warm unde...",B
5170,SURRATT BEAUTY,AU NATURELLE,Lipgloss,a universal neutral with a kiss of gold. ItÕs ...,C+
5171,CHANEL,SENSUEL (11),Lipgloss,a peachy brown with a very fine gold shimmer. ...,B
5172,NARS,BELLE DE JOUR,Lipstick,"a light-medium, yellowed peach with warm under...",D


In [5]:
def removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

In [6]:
clean = new_df['Review'].apply(removeNonAscii)

In [7]:
new_df['Review'] = clean

In [8]:
new_df.replace('-', ' ', regex=True, inplace=True)

In [9]:
new_df.sample(n=20)

,Brand,Name,Product,Review,Grade
1995,LAURA MERCIER,MODE TO MODERN,Palette,"a new, limited edition set featuring four, min...",A
4405,TOM FORD BEAUTY,RORY,Lipstick,a heavily gold shimmered peach with a metallic...,D
2211,NYX,STREET CRED,Lipstick,"a bright, medium pink coral with warm underton...",A
2337,MAKEUP GEEK,IN THE NUDE,Palette,"nine eyeshadows with four new, palette exclusi...",B+
533,BOBBI BROWN,VIBRANT VIOLET,Lipstick,"a bright, deep magenta with cool undertones an...",B
699,COLOURED RAINE,PARTY DRIP,Eyeshadow,"a vivid, medium fuchsia pink with strong, cool...",A
2348,MAYBELLINE,CHARMER,Lipgloss,"a muted, medium pink with warm undertones and ...",B+
584,TOO FACED,TF 20,Lipstick,"a deep, raspberry pink with cool, blue underto...",A+
2453,CHANEL,MEDIUM DEEP,Bronzer,"a medium brown with warm, yellow undertones an...",A
3078,CHANEL,CAMELIA DE CHANEL,Highlighter,"a light, golden beige with warm undertones and...",B+


In [10]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', 
                                   token_pattern="\\b[a-z][a-z]+\\b",
                                   lowercase=True,
                                   max_df = 0.6)
cv_data = count_vectorizer.fit_transform(new_df['Review'])
lsa_ = TruncatedSVD(n_components=90)
lsa_data = lsa_.fit_transform(cv_data)

In [15]:
new_vec = lsa_.transform(count_vectorizer.transform(['a new, limited edition neutral-hued palette that leans slightly more mauve/plum than overtly warm-toned. Of the three palettes launched in the Art Library collection, this has the most repeated shades (past releases and/or permanent shades). If you feel like a lot of matte eyeshadows are too powdery, you may you enjoy MAC’s formula, which tends to be firmer, a little stiffer and better for gradually building color with little to no fallout. Most of the mattes were blendable, but the darker shades–Sugar Plum and Dance in the Dark–could have been easier to soften along the edges. LES Artiste had some fallout and a tendency to emphasize my lid texture, so I’d recommend using fingertips or a wet brush to apply it, and I felt like Nude Model was overly slippery.']))
nn = NearestNeighbors(n_neighbors=1, metric='cosine')
nn.fit(lsa_data)
results = nn.kneighbors(new_vec, n_neighbors=5)


In [16]:
for i in range(len(results[1][0])):
        print(new_df.iloc[results[1][0][i]])

Brand                                             COLOUR POP
Name                                                     WET
Product                                              Palette
Review     a new, limited edition palette with six cooler...
Grade                                                     B+
Name: 22, dtype: object
Brand                                              KAT VON D
Name                                          SAINT + SINNER
Product                                              Palette
Review     a new, limited edition palette for the holiday...
Grade                                                     B+
Name: 2040, dtype: object
Brand                                                LANCOME
Name                                     AUDA[CITY] IN PARIS
Product                                              Palette
Review     a smokier set of shades that mixes cool and wa...
Grade                                                     C+
Name: 4762, dtype: object
Brand    

In [17]:
def get_nearest_prods():
    new_vec = lsa_.transform(count_vectorizer.transform([input()]))
    results = nn.kneighbors(new_vec, n_neighbors=5)
    rec_strings = []
    for i in range(len(results[1][0])):
        rec_strings.append(f"Try {new_df.iloc[results[1][0][i]]['Name']}, a {new_df.iloc[results[1][0][i]]['Product']} by {new_df.iloc[results[1][0][i]]['Brand']}.")
    return rec_strings

    

In [18]:
import pickle
pickle.dump(count_vectorizer, open('vectorizer.pkl', 'wb'))

In [19]:
pickle.dump(lsa_, open('dim_red.pkl', 'wb'))

In [20]:
pickle.dump(nn, open('model.pkl', 'wb'))

In [21]:
pickle.dump(new_df, open('reviews.pkl', 'wb'))